In [1]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class  # Mirror the matrix
    
    # Print the delta class matrix
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    
    # Calculate the delta score matrix
    delta_score_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, Δ_score, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_score_matrix[i, j] = Δ_score
            delta_score_matrix[j, i] = Δ_score  # Mirror the matrix
    
    # Print the delta score matrix
    print(f"Round {round_num} Delta Score Matrix:")
    print(delta_score_matrix)
    
    # Calculate the P_KS matrix
    p_ks_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, p_ks, _ = perform_differential_testing(predictions[i], predictions[j])
            p_ks_matrix[i, j] = p_ks
            p_ks_matrix[j, i] = p_ks  # Mirror the matrix
    
    # Print the P_KS matrix
    print(f"Round {round_num} P_KS Matrix:")
    print(p_ks_matrix)
    
    # Calculate the P_X2 matrix
    p_x2_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, _, p_x2 = perform_differential_testing(predictions[i], predictions[j])
            p_x2_matrix[i, j] = p_x2
            p_x2_matrix[j, i] = p_x2  # Mirror the matrix
    
    # Print the P_X2 matrix
    print(f"Round {round_num} P_X2 Matrix:")
    print(p_x2_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(p_x2_matrix[:, np.newaxis] - p_x2_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    #print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.8445333), ('loss', 0.5842217)]))])
Round 1 Delta Class Matrix:
[[   0. 8817. 8870. 8833. 8920. 8658. 8876. 8852. 8909. 8812.]
 [8817.    0. 2142. 2032. 1938. 1878. 1710. 2070. 2010. 1801.]
 [8870. 2142.    0. 2163. 2323. 2109. 2050. 2191. 2126. 2100.]
 [8833. 2032. 2163.    0. 2178. 2137. 1927. 2213. 2010. 2081.]
 [8920. 1938. 2323. 2178.    0. 1938. 1843. 2185. 2112. 1969.]
 [8658. 1878. 2109. 2137. 1938.    0. 1790. 2030. 2058. 1867.]
 [8876. 1710. 2050. 1927. 1843. 1790.    0. 1920. 1904. 1726.]
 [8852. 2070. 2191. 2213. 2185. 2030. 1920.    0. 2227. 1947.]
 [8909. 2010. 2126. 2010. 2112. 2058. 1904. 2227.    0. 1975.]
 [8812. 1801. 2100. 2081. 1969. 1867. 1726. 1947. 1975.    0.]]
Round 1 Delta Score Matrix:
[[    0. 99980. 99987. 99984. 99987. 99987. 99988. 99985. 99978. 99984.]
 [99980.   

Here’s how we can show the client with the highest average distance after each test in each round:

In [2]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class  # Mirror the matrix
    
    # Print the delta class matrix
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    
    # Calculate the distance matrix for delta class
    distance_matrix_class = np.linalg.norm(delta_class_matrix[:, np.newaxis] - delta_class_matrix, axis=2)
    
    # Calculate and print average distances for delta class
    average_distances_class = np.mean(distance_matrix_class, axis=1)
    print(f"Round {round_num} Average distances (Delta Class):")
    print(average_distances_class)
    
    # Identify the client with the highest average distance for delta class
    max_distance_client_class = np.argmax(average_distances_class)
    print(f"Round {round_num}, Client with highest average distance (Delta Class): {max_distance_client_class}")
    
    # Calculate the delta score matrix
    delta_score_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, Δ_score, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_score_matrix[i, j] = Δ_score
            delta_score_matrix[j, i] = Δ_score  # Mirror the matrix
    
    # Print the delta score matrix
    print(f"Round {round_num} Delta Score Matrix:")
    print(delta_score_matrix)
    
    # Calculate the distance matrix for delta score
    distance_matrix_score = np.linalg.norm(delta_score_matrix[:, np.newaxis] - delta_score_matrix, axis=2)
    
    # Calculate and print average distances for delta score
    average_distances_score = np.mean(distance_matrix_score, axis=1)
    print(f"Round {round_num} Average distances (Delta Score):")
    print(average_distances_score)
    
    # Identify the client with the highest average distance for delta score
    max_distance_client_score = np.argmax(average_distances_score)
    print(f"Round {round_num}, Client with highest average distance (Delta Score): {max_distance_client_score}")
    
    # Calculate the P_KS matrix
    p_ks_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, p_ks, _ = perform_differential_testing(predictions[i], predictions[j])
            p_ks_matrix[i, j] = p_ks
            p_ks_matrix[j, i] = p_ks  # Mirror the matrix
    
    # Print the P_KS matrix
    print(f"Round {round_num} P_KS Matrix:")
    print(p_ks_matrix)
    
    # Calculate the distance matrix for P_KS
    distance_matrix_ks = np.linalg.norm(p_ks_matrix[:, np.newaxis] - p_ks_matrix, axis=2)
    
    # Calculate and print average distances for P_KS
    average_distances_ks = np.mean(distance_matrix_ks, axis=1)
    print(f"Round {round_num} Average distances (P_KS):")
    print(average_distances_ks)
    
    # Identify the client with the highest average distance for P_KS
    max_distance_client_ks = np.argmax(average_distances_ks)
    print(f"Round {round_num}, Client with highest average distance (P_KS): {max_distance_client_ks}")
    
        # Calculate the P_X2 matrix
    p_x2_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, _, p_x2 = perform_differential_testing(predictions[i], predictions[j])
            p_x2_matrix[i, j] = p_x2
            p_x2_matrix[j, i] = p_x2  # Mirror the matrix
    
    # Print the P_X2 matrix
    print(f"Round {round_num} P_X2 Matrix:")
    print(p_x2_matrix)
    
    # Calculate the distance matrix for P_X2
    distance_matrix_x2 = np.linalg.norm(p_x2_matrix[:, np.newaxis] - p_x2_matrix, axis=2)
    
    # Calculate and print average distances for P_X2
    average_distances_x2 = np.mean(distance_matrix_x2, axis=1)
    print(f"Round {round_num} Average distances (P_X2):")
    print(average_distances_x2)
    
    # Identify the client with the highest average distance for P_X2
    max_distance_client_x2 = np.argmax(average_distances_x2)
    print(f"Round {round_num}, Client with highest average distance (P_X2): {max_distance_client_x2}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.84113336), ('loss', 0.58818406)]))])
Round 1 Delta Class Matrix:
[[   0. 8989. 8902. 8994. 8827. 9090. 9048. 8881. 8894. 9103.]
 [8989.    0. 1887. 1892. 2069. 2064. 2050. 2107. 2114. 1804.]
 [8902. 1887.    0. 1700. 1987. 1704. 1897. 1823. 2027. 1720.]
 [8994. 1892. 1700.    0. 1986. 1848. 2015. 1940. 1999. 1705.]
 [8827. 2069. 1987. 1986.    0. 2211. 2227. 2167. 2213. 1898.]
 [9090. 2064. 1704. 1848. 2211.    0. 2035. 1973. 2207. 1868.]
 [9048. 2050. 1897. 2015. 2227. 2035.    0. 2115. 2193. 1941.]
 [8881. 2107. 1823. 1940. 2167. 1973. 2115.    0. 2176. 1959.]
 [8894. 2114. 2027. 1999. 2213. 2207. 2193. 2176.    0. 1981.]
 [9103. 1804. 1720. 1705. 1898. 1868. 1941. 1959. 1981.    0.]]
Round 1 Average distances (Delta Class):
[21132.72494126  4627.23935819  4517.51738868  4539.96226251
  4725.2

It looks like the P_KS metric is not performing as expected in identifying the bugged client, while the other metrics (Δ_class, Δ_score, and P_X2) are correctly identifying the bugged client.

The Kolmogorov-Smirnov (KS) test is used to compare the distributions of two samples. If the distributions are very similar, the KS test might not be sensitive enough to detect the differences caused by the bug in the model. This could explain why the P_KS metric is not identifying the bugged client correctly

To improve detection, let's modify the approach by combining multiple metrics (Δ_class, Δ_score, P_KS, and P_X2) for the distance calculation. Here's the complete implementation with the requested changes:

In [3]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    delta_score_matrix = np.zeros((num_clients, num_clients))
    p_ks_matrix = np.zeros((num_clients, num_clients))
    p_x2_matrix = np.zeros((num_clients, num_clients))
    
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, Δ_score, P_KS, P_X2 = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class
            delta_score_matrix[i, j] = Δ_score
            delta_score_matrix[j, i] = Δ_score
            p_ks_matrix[i, j] = P_KS
            p_ks_matrix[j, i] = P_KS
            p_x2_matrix[i, j] = P_X2
            p_x2_matrix[j, i] = P_X2
    
    # Print the matrices
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    print(f"Round {round_num} Delta Score Matrix:")
    print(delta_score_matrix)
    print(f"Round {round_num} P_KS Matrix:")
    print(p_ks_matrix)
    print(f"Round {round_num} P_X2 Matrix:")
    print(p_x2_matrix)
    
    # Combine multiple matrices for the final distance calculation
    combined_matrix = delta_class_matrix + delta_score_matrix + p_ks_matrix + p_x2_matrix
    distance_matrix = np.linalg.norm(combined_matrix[:, np.newaxis] - combined_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Combined Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.8483667), ('loss', 0.56931394)]))])
Round 1 Delta Class Matrix:
[[   0. 9131. 8997. 9197. 9190. 9122. 8872. 9178. 9049. 8722.]
 [9131.    0. 1984. 2165. 1985. 1875. 1833. 1856. 1858. 1919.]
 [8997. 1984.    0. 2253. 2074. 2045. 1960. 1990. 2084. 1985.]
 [9197. 2165. 2253.    0. 2267. 2157. 2076. 2185. 2170. 2112.]
 [9190. 1985. 2074. 2267.    0. 1894. 1902. 1941. 2007. 1990.]
 [9122. 1875. 2045. 2157. 1894.    0. 1793. 1878. 1826. 1916.]
 [8872. 1833. 1960. 2076. 1902. 1793.    0. 1890. 1765. 1811.]
 [9178. 1856. 1990. 2185. 1941. 1878. 1890.    0. 1930. 1912.]
 [9049. 1858. 2084. 2170. 2007. 1826. 1765. 1930.    0. 1933.]
 [8722. 1919. 1985. 2112. 1990. 1916. 1811. 1912. 1933.    0.]]
Round 1 Delta Score Matrix:
[[    0. 99945. 99938. 99929. 99936. 99944. 99925. 99954. 99943. 99936.]
 [99945.  

In [5]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=8):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    delta_score_matrix = np.zeros((num_clients, num_clients))
    p_ks_matrix = np.zeros((num_clients, num_clients))
    p_x2_matrix = np.zeros((num_clients, num_clients))
    
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, Δ_score, P_KS, P_X2 = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class
            delta_score_matrix[i, j] = Δ_score
            delta_score_matrix[j, i] = Δ_score
            p_ks_matrix[i, j] = P_KS
            p_ks_matrix[j, i] = P_KS
            p_x2_matrix[i, j] = P_X2
            p_x2_matrix[j, i] = P_X2
    
    # Print the matrices
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    print(f"Round {round_num} Delta Score Matrix:")
    print(delta_score_matrix)
    print(f"Round {round_num} P_KS Matrix:")
    print(p_ks_matrix)
    print(f"Round {round_num} P_X2 Matrix:")
    print(p_x2_matrix)
    
    # Combine multiple matrices for the final distance calculation
    combined_matrix = delta_class_matrix + delta_score_matrix + p_ks_matrix + p_x2_matrix
    distance_matrix = np.linalg.norm(combined_matrix[:, np.newaxis] - combined_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Combined Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.8436), ('loss', 0.5868787)]))])
Round 1 Delta Class Matrix:
[[   0. 8883. 8887. 8991. 9025. 9041. 9248. 9103. 8913. 8822.]
 [8883.    0. 1882. 2213. 1994. 2027. 2309. 1993. 2163. 1991.]
 [8887. 1882.    0. 1970. 1763. 1849. 2196. 1785. 1963. 1787.]
 [8991. 2213. 1970.    0. 2059. 2124. 2354. 2075. 2172. 2077.]
 [9025. 1994. 1763. 2059.    0. 1817. 2205. 1779. 2014. 1955.]
 [9041. 2027. 1849. 2124. 1817.    0. 2136. 1871. 2105. 1885.]
 [9248. 2309. 2196. 2354. 2205. 2136.    0. 2187. 2400. 2233.]
 [9103. 1993. 1785. 2075. 1779. 1871. 2187.    0. 2054. 1999.]
 [8913. 2163. 1963. 2172. 2014. 2105. 2400. 2054.    0. 2122.]
 [8822. 1991. 1787. 2077. 1955. 1885. 2233. 1999. 2122.    0.]]
Round 1 Delta Score Matrix:
[[    0. 99909. 99911. 99946. 99919. 99933. 99921. 99913. 99929. 99923.]
 [99909.     0

 Handling of Multi-Metric Data
Average Distance Approach: In your implementation, combining multiple metrics (e.g., Δ_class, Δ_score, P_KS, and P_X2) into a single average distance gives a comprehensive view of client behavior. This multi-metric perspective increases the robustness of detection.
DBSCAN: While DBSCAN can handle multi-dimensional data, it may struggle to integrate the information across different metrics effectively, especially if the scales of the metrics vary significantly. Distance measures must be defined appropriately, which could complicate the implementation.